<a href="https://colab.research.google.com/github/Ivyson/Neural-Network-XOR/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import numpy as np

class NeuralNetwork():
    def __init__(self, input_size, hidden_nodes, output_size, learning_rate=0.1):
        """
        :param input_size: Number of input neurons
        :param hidden_nodes: List specifying number of neurons in each hidden layer
        :param output_size: Number of output neurons
        :param learning_rate: Learning rate for weight updates

        """
        self.input_size = input_size
        self.hidden_nodes = hidden_nodes  # List specifying neurons per hidden layer
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Define the architecture: input layer → hidden layers → output layer
        layer_sizes = [input_size] + hidden_nodes + [output_size]

        # Initialize weights and biases dynamically
        self.weights = [np.random.rand(layer_sizes[i], layer_sizes[i+1]) - 0.5 for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.rand(layer_sizes[i+1]) - 0.5 for i in range(len(layer_sizes) - 1)]

    def sigmoid(self, x):
      return 1 / (1 + np.exp(-x))


    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def feedForward(self, inputs):
        # Forward propagation through all layers.......
        self.layers = [inputs]  # Store activations of all layers
        for i in range(len(self.weights)):
            inputs = self.sigmoid(np.dot(inputs, self.weights[i]) + self.biases[i])
            self.layers.append(inputs)  # Save outputs of [i+1] layer for backpropagation
        return inputs


    # Back Prop, to update the weights and Biases of the nueral network
    def backpropagation(self, target_output):
        errors = [target_output - self.layers[-1]]  # Output layer error
        deltas = [errors[0] * self.sigmoid_derivative(self.layers[-1])]  # Output layer delta

        # Get Dltas For each hidden layer in reverse order
        for i in range(len(self.hidden_nodes), 0, -1):
            errors.insert(0, np.dot(deltas[0], self.weights[i].T))  # Error of previous layer
            deltas.insert(0, errors[0] * self.sigmoid_derivative(self.layers[i]))  # Delta, previous layer

        # Update weights and biases
        for i in range(len(self.weights)):
            self.weights[i] += np.dot(self.layers[i].reshape(-1, 1), deltas[i].reshape(1, -1)) * self.learning_rate
            self.biases[i] += deltas[i] * self.learning_rate

    def train(self, X, y, epochs=10000):
        for epoch in range(epochs):
            total_loss = 0
            for i in range(len(X)):
                self.feedForward(X[i])
                self.backpropagation(y[i])
                total_loss += np.sum(np.abs(y[i] - self.layers[-1]))

            if epoch % 3000 == 0:
                print(f"Epoch {epoch}, Loss: {(total_loss / len(X)):.2f}")

    def predict(self, X):
        return [self.feedForward(x) for x in X]


    def save_model(self, filename):
        with open(filename, 'wb') as file:
            np.save(file, self.input_size)
            np.save(file, self.hidden_nodes)
            np.save(file, self.output_size)


            # Save all weights and biases
            for weight in self.weights:
                np.save(file, weight)

            for bias in self.biases:
                np.save(file, bias)


    def Load_Model(self, filename):
      # Open the file in read mode
      with open(filename, 'rb') as file:
          self.input_size = np.load(file)
          self.hidden_nodes = np.load(file)
          self.output_size = np.load(file)
          # Size of the weights = [len(inputs)*Hidden[0]][Hidden[0]*]
          self.weights = []
          self.biases = []
          size = [self.input_size] + self.hidden_nodes + [self.output_size]
          self.weights = [np.load(file) for _ in range(len(size) + 1)]
          self.biases = [np.load(file) for _ in range(len(size) + 1)]
          # print(f'Biases : {self.biases}')
          # print(f'Weights : {self.weights}')
          # print(f'Input Size : {self.input_size}')
          # print(f'Hidden Nodes : {self.hidden_nodes}')
          # print(f'Output Size : {self.output_size} ')
          print(f'Model : {filename} has been Loaded successfully')


# OR dataset
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])
#  Desired Output/ Target Output
y = np.array([
    [0],
    [1],
    [1],
    [1]
])

# Create A Nueral Network with 2 inputs, and 2 Hidden Layers with nodes each,
nn = NeuralNetwork(input_size=2, hidden_nodes=[20], output_size=1, learning_rate=0.2)
nn.train(X, y, epochs=10000)

"""
The Model is too small and the learning rate is pretty quick
So, The 50 Thousands epochs are not tha much of a deal,
ever since the model has to solve a basic problem

"""
nn.save_model('model.txt')


Epoch 0, Loss: 0.62
Epoch 3000, Loss: 0.02
Epoch 6000, Loss: 0.01
Epoch 9000, Loss: 0.01


In [93]:
nn.Load_Model('model.txt')